# Quantstar - Revolutionizing Satellite Operations

## Imports

In [ ]:
from mqt import ddsim
import numpy as np

from mqt.problemsolver import utils
from mqt.problemsolver.utils import  QAOA, W_QAOA

from qiskit import Aer, execute
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters.quadratic_program_to_qubo import (
    QuadraticProgramToQubo,
)
from qiskit.algorithms.optimizers import L_BFGS_B, ADAM, SPSA

import matplotlib.pyplot as plt

import warnings
import time

## Create arbitrary Satellite Problem Instance

In [ ]:
num_requests = 5
ac_reqs = utils.init_random_acquisition_requests(num_requests)
utils.plot_acqisition_requests(ac_reqs)

## Create Docplex Model and Convert it to QUBO Formulation

In [ ]:
penalty = None
mdl = utils.create_satellite_doxplex(ac_reqs)
converter, qubo = utils.convert_docplex_to_qubo(mdl, penalty=penalty)
print(qubo.prettyprint())
print(qubo.to_ising())
converter, qubo = utils.convert_docplex_to_qubo(mdl)
print(qubo.to_ising())
for elem in qubo.to_ising():
    print(elem, type(elem))

## QAOA Solution Implementation

In [ ]:
def solve_using_qaoa(qubo) -> bool:
    qaoa = QAOA(QAOA_params={"reps":3, "optimizer": SPSA(maxiter=100)})
    qc_qaoa, res_qaoa = qaoa.get_solution(qubo)

    num_shots = 10000
    backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
    job = execute(qc_qaoa, backend, shots=num_shots)
    qaoa_counts = job.result().get_counts(qc_qaoa)

    probs={}
    for key in qaoa_counts:
        bin_val = bin(int(key, 2))[2:].zfill(len(qubo.variables))
        state = converter.interpret(np.array(list(bin_val), dtype=int))
        probs[bin_val] = qaoa_counts[key]/num_shots

    most_likely_eigenstate_qaoa = utils.sample_most_likely(probs)
    most_likely_eigenstate_qaoa = np.array([int(s) for s in most_likely_eigenstate_qaoa], dtype=float)
    if utils.check_solution(ac_reqs, most_likely_eigenstate_qaoa):
        solution = most_likely_eigenstate_qaoa
        value = utils.calc_sol_value(ac_reqs, most_likely_eigenstate_qaoa)
        return solution, value
    else:
        return False

In [ ]:
ac_reqs = utils.init_random_acquisition_requests(5)
mdl = utils.create_satellite_doxplex(ac_reqs)
converter, qubo = utils.convert_docplex_to_qubo(mdl)
qaoa = QAOA(QAOA_params={"reps":1, "optimizer": SPSA(maxiter=1)})
qc_qaoa, res_qaoa = qaoa.get_solution(qubo)
print(qubo.to_ising())
print(qubo)
print(np.array(qubo.to_ising()[0].primitive.coeffs, dtype=float))
print(qubo.to_ising()[1])
print(qaoa.ansatz.decompose(reps=2))

### Execution

In [ ]:
num_runs = 5
max_problem_size_qaoa = 10
res_all_qaoa_times = []
res_all_qaoa = []
for i in range (2, max_problem_size_qaoa):
    ac_reqs = utils.init_random_acquisition_requests(i)
    mdl = utils.create_satellite_doxplex(ac_reqs)
    converter, qubo = utils.convert_docplex_to_qubo(mdl)
    print(i)
    res_tmp = []
    res_tmp_time = []
    success = 0
    for j in range(num_runs):
        start_time = time.time()
        if solve_using_qaoa(qubo):
            success += 1
        res_tmp_time.append(time.time()-start_time)
    res_all_qaoa.append(success/num_runs)
    res_all_qaoa_times.append(sum(res_tmp_time)/num_runs)

## W-QAOA Solution

In [ ]:
def solve_using_w_qaoa(qubo) -> bool:
    wqaoa = W_QAOA()
    qc_wqaoa, res_wqaoa = wqaoa.get_solution(qubo)
    return res_wqaoa

### Execution

In [ ]:
num_runs = 5
max_problem_size_wqaoa = 10
res_all_wqaoa_times = []
res_all_wqaoa = []
for i in range (2, max_problem_size_wqaoa):
    print(i)
    ac_reqs = utils.init_random_acquisition_requests(i)
    mdl = utils.create_satellite_doxplex(ac_reqs)
    converter, qubo = utils.convert_docplex_to_qubo(mdl)
    res_tmp = []
    res_tmp_time = []
    success = 0
    for j in range(num_runs):
        start_time = time.time()
        res_w_qaoa = solve_using_w_qaoa(qubo)
        if res_w_qaoa.status.value==0:
            success += 1
        res_tmp_time.append(time.time()-start_time)
    res_all_wqaoa.append(success/num_runs)
    res_all_wqaoa_times.append(sum(res_tmp_time)/num_runs)

## Generate Plots

In [ ]:
plt.plot(range(2, max_problem_size_qaoa), res_all_qaoa, label="QAOA")
plt.plot(range(2, max_problem_size_wqaoa), res_all_wqaoa, label="W-QAOA")
plt.title("Average Success Probability per Problem Size")
plt.xlabel("Problem Size")
plt.ylabel("Success Probability")
plt.legend()
plt.show()

plt.plot(range(2, max_problem_size_qaoa), res_all_qaoa_times, label="QAOA")
plt.plot(range(2, max_problem_size_wqaoa), res_all_wqaoa_times, label="W-QAOA")
plt.title("Average Runtime per Problem Size")
plt.xlabel("Problem Size")
plt.ylabel("Runtime in s")
plt.legend()
plt.show()